# Preamble

Ignore the errors raised because of nb-black

In [ ]:
cd ../

In [7]:
pwd

'/Users/ashkanaleali/w/infer_struct'

time: 2.4 ms (started: 2021-03-30 22:50:53 -05:00)


In [8]:
%load_ext lab_black
%matplotlib inline
%load_ext autoreload
%autoreload 2
%load_ext autotime

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.92 ms (started: 2021-03-30 22:51:28 -05:00)


## Imports

In [162]:
import os
import numpy as np
import pandas as pd
import networkx as nx

time: 7.84 ms (started: 2021-03-31 01:38:22 -05:00)


In [109]:
from infer_struct.data import PowerGridSimpleExample

time: 4.13 ms (started: 2021-03-31 00:12:57 -05:00)


# Data

This notebook is just to see how to load the data Amir has provided.

In [148]:
node_number = 9
data_dir = "../data/Network_raw_data_Ashkan/"
example_id = f"{node_number}_nodes"

time: 4.35 ms (started: 2021-03-31 00:41:04 -05:00)


In [149]:
ex = PowerGridSimpleExample(node_number)

time: 31.2 ms (started: 2021-03-31 00:41:04 -05:00)


+ There are 2 $nxn$ matrices and 6 vectors for each example. The vectors have length < $n$ and the missing elements are treated as zero.
+ Each vector is given by a tuple where the first element is the index of the element -- sparse matrix representation?


The PowerGridSimpleExample do some normalizations.

Let's add network data stcruture to it

In [166]:
adjacency_matrix = ((ex.Ybus_mag + ex.Ybus_theta) > 0).astype(int)
g = nx.Graph(adjacency_matrixb)

time: 4.58 ms (started: 2021-03-31 01:39:21 -05:00)


In [168]:
nx.set_edge_attributes()

EdgeView([(0, 3), (1, 1), (1, 6), (2, 2), (2, 8), (3, 3), (3, 4), (3, 5), (4, 4), (4, 6), (5, 5), (5, 8), (6, 6), (6, 7), (7, 7), (7, 8), (8, 8)])

time: 3.82 ms (started: 2021-03-31 01:39:30 -05:00)
